In [17]:
import pandas as pd
import numpy as np
import cPickle as pickle

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [2]:
prop = pickle.load(open("../../datos/Data.p","rb"))

# Nos quedamos con la parte del set de datos que nos importa

In [3]:
p = prop[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos','Cant_ParadasTransporte','price_aprox_usd']]
p.dropna(inplace = True)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Separamos la Latitud-Longitud

In [4]:
p['lat'] = p.apply(lambda row: row[5][0], axis=1)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
p['lon'] = p.apply(lambda row: row[5][1], axis=1)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
p = p[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos','Cant_ParadasTransporte','price_aprox_usd']]

# Asignamos un valor numerico al tipo de propiedad

In [7]:
def asignarPT( p ):
    if p == 'apartment':
        return 0
    if p == 'house':
        return 1
    if p == 'store':
        return 2
    return 3

In [8]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

# Asignamos un valor a los barrios

In [9]:
def asignarPV(name, dic):
    return dic[name]

In [10]:
hoodGroups = p[["place_name", "price_usd_per_m2"]]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("price_usd_per_m2", "mean"), ascending=False)

In [11]:
HGdics = {}
precio_ant = 0
val_ant = 0
for row in hoodGroups.iterrows():
    name = row[1][0]
    price = row[1][1]
    val = val_ant + (precio_ant/price)*10
    HGdics[name] = val
    precio_ant = price
    val_ant = val

In [12]:
p["place_value"] = p.apply(lambda x: asignarPV(x[1], HGdics), axis=1)

In [13]:
p = p[['property_type','surface_total_in_m2',\
          'lat','lon','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value', 'price_aprox_usd']]

# Separamos los datos

In [14]:
Xtrn, Xtest, Ytrn, Ytest = train_test_split(p[['property_type','surface_total_in_m2',
        'place_value']],p[['price_aprox_usd']],test_size=0.2)

# Intento hacer una pasada con parametros random

In [15]:
ada = AdaBoostRegressor()
ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

In [16]:
error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

203262202225.44461

# Intentamos aproximar un poco los parametros

In [87]:
#para todos
lista_estimadores = [x for x in xrange(1,200)]
lista_loss = ['linear', 'square', 'exponential']

#Gradient Boosting
lista_loss2 = ['ls', 'lad', 'huber', 'quantile']
lista_e = [x for x in xrange(1,200, 5)]
lista_depth = [x for x in xrange(1,10)]
lista_criterios = ['friedman_mse', 'mse', 'mae']
lista_min_split = [0.2,0.4,0.6,0.8,1.0,2,3,4,5,6,7,8,9,10]
lista_min_leaf = [x for x in xrange(1,10)]
lista_min_weight = [0.0,0.1,0.2,0.3,0.4,0.5]
lista_max_features = ['auto', 'sqrt', 'log2', None]
lista_max_leaf = [x for x in xrange(1,10), None]

In [77]:
aux = Ytest.copy()

#Inicializo la instancia del clasificador
gb = GradientBoostingRegressor(n_estimators=1, criterion='mse')

#Inicializo la instancia del algoritmo
ada = AdaBoostRegressor( n_estimators = 1, base_estimator = gb, loss = 'linear')
ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux['predict'] = ada.predict(Xtest)

In [53]:
a = GradientBoostingClassifier(n_estimators=100, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1,\
                               min_weight_fraction_leaf=0.0, max_depth=3, max_leaf_nodes=None,max_features = None) 

In [88]:
#Hago una corrida con Gradient Boosting
res = []
for e in lista_estimadores:
    for l in lista_loss:
        for l2 in lista_loss2:
            for egb in lista_e:
                for d in lista_depth:
                    for c in lista_criterios:
                        for mins in lista_min_split:
                            for minl in lista_min_leaf:
                                for minw in lista_min_weight:
                                    for maxf in lista_max_features:
                                        aux = Ytest.copy()

                                        #Inicializo la instancia del clasificador
                                        gb = GradientBoostingRegressor(n_estimators=egb, criterion=c,\
                                        min_samples_split=mins, min_samples_leaf=minl ,\
                                        min_weight_fraction_leaf=minw, loss = l2,\
                                        max_depth=d, max_features = maxf)

                                        params = (egb,c,l2,mins,minl,minw,d,maxf)
                                        #Inicializo la instancia del algoritmo
                                        ada = AdaBoostRegressor( n_estimators = e, base_estimator = gb, loss = 'linear')
                                        ada.fit(Xtrn, Ytrn['price_aprox_usd'])
                                        aux['predict'] = ada.predict(Xtest)

                                        error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])

                                        res.append( (error, e, 'GradientBossting', l, params) )

KeyboardInterrupt: 

In [93]:
r = sorted(res, key=lambda x: x[0])

In [99]:
#aca podemos ver una busqueda aproximada de los parametros
r

[(141105552380.71796,
  1,
  'GradientBossting',
  'linear',
  (1, 'mse', 'ls', 10, 3, 0.3, 1, 'auto')),
 (141174098257.77689,
  1,
  'GradientBossting',
  'linear',
  (1, 'friedman_mse', 'ls', 4, 4, 0.3, 1, None)),
 (141182419382.61057,
  1,
  'GradientBossting',
  'linear',
  (1, 'mse', 'ls', 5, 4, 0.1, 1, None)),
 (141184084228.15607,
  1,
  'GradientBossting',
  'linear',
  (1, 'friedman_mse', 'ls', 8, 7, 0.1, 1, None)),
 (141188280179.66409,
  1,
  'GradientBossting',
  'linear',
  (1, 'friedman_mse', 'ls', 6, 6, 0.3, 1, None)),
 (141188726246.83066,
  1,
  'GradientBossting',
  'linear',
  (1, 'mse', 'ls', 0.8, 9, 0.2, 1, 'auto')),
 (141189141281.22485,
  1,
  'GradientBossting',
  'linear',
  (1, 'mse', 'ls', 2, 7, 0.0, 1, None)),
 (141190259972.43655,
  1,
  'GradientBossting',
  'linear',
  (1, 'friedman_mse', 'ls', 9, 7, 0.0, 1, 'sqrt')),
 (141190494423.62631,
  1,
  'GradientBossting',
  'linear',
  (1, 'friedman_mse', 'ls', 0.6, 1, 0.1, 1, 'auto')),
 (141197163463.50601,
  

In [ ]:
#Decision Tree
lista_criterios = ['friedman_mse', 'mse', 'mae']
lista_splitter = ['best','random']
lista_min_split = [0.2,0.4,0.6,0.8,1.0,2,3,4,5,6,7,8,9,10]
lista_min_leaf = [x for x in xrange(1,10)]
lista_min_weight = [0.0,0.1,0.2,0.3,0.4,0.5]
lista_max_features = ['auto', 'sqrt', 'log2', None]
lista_max_leaf = [x for x in xrange(1,10), None]
lista_depth = [x for x in xrange(1,10)]

# Pruebo con un greed search

In [36]:
def Grid_Search(Xtrn, Ytrn):
    estimator = GradientBoostingRegressor(criterion='mse',loss = 'ls')
    
    param_grid = { 
            "n_estimators"      : [1,2,3,4,5,95,96,97,98,99,100,110,115,130,140,150],
            "max_features"      : ["auto",'sqrt', 'log2', None],
            "min_samples_split" : [2,3,4,5,6,7,8,9,10] ,
            "min_samples_leaf" : [1,2,3,4,5,6,7,8,9,10]
            }

    grid = GridSearchCV(estimator, param_grid, cv=5)

    grid.fit(Xtrn, Ytrn)

    return grid.best_score_ , grid.best_params_

In [37]:
resultados = []

for i in range(1):
    
    #cross validation
    Xtrn, Xtest, Ytrn, Ytest = train_test_split(p[['property_type','surface_total_in_m2',
        'lat','lon','Cant_LocalesGastronomicos','Cant_ParadasTransporte',
        'place_value']],p[['price_aprox_usd']],test_size=0.2)
    
    #grid search
    best_score, best_params = Grid_Search(Xtrn, Ytrn['price_aprox_usd'])
    
    #prueba de los mejores resultados
    estimator = GradientBoostingRegressor(criterion='mse',loss = 'ls').set_params(**best_params)
    estimator.fit(Xtrn,Ytrn['price_aprox_usd'])
    
    #prediccion 
    pred = mean_squared_error(Ytest['price_aprox_usd'], estimator.predict(Xtest))
    print ('RMSE_Price', pred)

    resultados.append((pred, best_score, best_params))

KeyboardInterrupt: 